In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import re

# СЫРЫЕ ДАННЫЕ из create_chart вызовов (скопируйте из истории)
TCO_RAW_DATA = {
    "Статья расходов": [
        "Аренда/покупка сервера в месяц", "Лицензии ПО", "Электричество/охлаждение",
        "Администрирование (чел-часы)", "Резервное копирование", "Миграция/развертывание VM",
        "Высокая доступность/HA кластер", "ИТОГО в год", "ИТОГО за 3 года"
    ],
    "Dediki (год)": [
        "$600 x 12 = $7,200", "$0-2,000", "$2,000-3,000", "400 часов x $50 = $20,000",
        "$1,500-3,000", "$3,000-5,000", "$3,000-8,000", "$36,700-41,200", "$110,000-123,600"
    ],
    "Proxmox на сервере (год)": [
        "$400 x 12 = $4,800", "$0", "$1,200-1,800", "300 часов x $50 = $15,000",
        "$500 (Proxmox Backup Server)", "$500 (быстро через Proxmox)", "$400 (2 node кластер)",
        "$22,500-27,300", "$67,500-81,900"
    ]
}

# Парсер из агента
def parse_cost(cost_str):
    numbers = re.findall(r'\d+(?:,\d+)*', cost_str.replace('$', ''))
    if not numbers:
        return 0
    nums = [int(n.replace(',', '')) for n in numbers]
    if len(nums) >= 2 and '-' in cost_str:
        return (nums[0] + nums[1]) / 2
    return nums[0]

# Преобразование сырых данных в числовые
df_tco = pd.DataFrame(TCO_RAW_DATA)
df_tco['Dediki_parsed'] = df_tco['Dediki (год)'].apply(parse_cost)
df_tco['Proxmox_parsed'] = df_tco['Proxmox на сервере (год)'].apply(parse_cost)

# Создание дашборда
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Сравнение затрат по статьям', 'Экономия Proxmox vs Dediki', 
                    'TCO за 3 года', 'Структура затрат'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# График 1: Сравнение по статьям
fig.add_trace(
    go.Bar(name='Dediki', x=df_tco['Статья расходов'], y=df_tco['Dediki_parsed'], 
           marker_color='crimson'), row=1, col=1
)
fig.add_trace(
    go.Bar(name='Proxmox', x=df_tco['Статья расходов'], y=df_tco['Proxmox_parsed'], 
           marker_color='forestgreen'), row=1, col=1
)

# График 2: Экономия
savings = df_tco['Dediki_parsed'] - df_tco['Proxmox_parsed']
fig.add_trace(
    go.Bar(x=df_tco['Статья расходов'], y=savings, marker_color='gold',
           name='Экономия'), row=1, col=2
)

# График 3: TCO за 3 года (только итоговые строки)
tco_summary = df_tco[df_tco['Статья расходов'].str.contains('ИТОГО')]
fig.add_trace(
    go.Pie(labels=tco_summary['Статья расходов'], values=tco_summary['Dediki_parsed'],
           name='Dediki TCO'), row=2, col=1
)
fig.add_trace(
    go.Pie(labels=tco_summary['Статья расходов'], values=tco_summary['Proxmox_parsed'],
           name='Proxmox TCO'), row=2, col=1
)

fig.update_layout(height=800, title_text="Dediki vs Proxmox: Полный дашборд TCO")
fig.show()
